In [1]:
import os
import pandas as pd
import calendar
import datetime
import monpa
from monpa import utils
import re
import numpy as np
import copy
os.chdir('/Users/arthur/Desktop/factor_investment')
# os.chdir('/Users/arthur/Desktop/Event_Driven_Stock_Prediction_using_Deep_Learning')

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [2]:
# 我們從stopwords_zh.txt這個檔案中匯入繁體中文的停用詞
with open('./202202_Intern/stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines()
file.close()

with open('./202202_Intern/NTUSD_negative_unicode.txt', 'r') as file:
    nega_words = file.read().splitlines()
file.close()

with open('./202202_Intern/NTUSD_positive_unicode.txt', 'r') as file:
    posi_words = file.read().splitlines()
file.close()

# 這個function用來將字串以  **正則化**  處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

def splitSentence(content):
    sentence_list = utils.short_sentence(content)
    tokenStr = []
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence)
        tokenStr += tokens
    return tokenStr

def calSen(content):
    sentence_list = utils.short_sentence(content)
    tokenStr = []
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence)
        tokenStr += tokens
        
    senScore = 0
    for i in tokenStr:
        if i in posi_words:
            senScore += 1
        elif i in nega_words:
            senScore -= 1
    return senScore

def find_key(dic, value):
    for keys, values in dic.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
        if value == values:
            return(keys)
            break

def trend(value):
    if value > 0:
        return 1
    elif value < 0:
        return -1
    else:
        return 0

In [3]:
# monpa.load_userdict("./202202_Intern/stock_abbr.txt")

In [4]:
for i in range(2013, 2015):
    for j in range(1,13):
        try:
            pd.read_csv('./202202_Intern/SegmentedData/'+str(i)+str(j)+'_cnyesnews.csv')
        except:
            print(i, j)
            try:
                df_news_1 = pd.read_csv('./202202_Intern/RawData/'+str(i)+str(j)+'1_cnyesnews.csv')[['date', 'title', 'content']]
                df_news_2 = pd.read_csv('./202202_Intern/RawData/'+str(i)+str(j)+'2_cnyesnews.csv')[['date', 'title', 'content']]
                df_news = pd.concat([df_news_1, df_news_2]).reset_index(drop=True)
                df_news.date = pd.to_datetime(df_news.date).dt.date
                
#                 for k in range(df_news.shape[0]):
#                     try:
#                         df_news.loc[k, 'title_score'] = calSen(df_news.loc[k, 'title'])
#                     except:
#                         df_news.loc[k, 'title_score'] = 0
#                     try:
#                         df_news.loc[k, 'content_score'] = calSen(df_news.loc[k, 'content'])
#                     except:
#                         df_news.loc[k, 'content_score'] = 0
#                     if (k % 100) == 0:
#                         print(k, ' / ', df_news.shape[0])
                df_news['all'] = df_news['title']+df_news['content']
                df_news['all'] = df_news['all'].fillna('')
                df_news['segmented'] = df_news['all'].apply(lambda x : splitSentence(x))
                df_news = df_news[['date', 'segmented']] # should add 'all'!!!!!

                df_news.to_csv('./202202_Intern/SegmentedData/'+str(i)+str(j)+'_cnyesnews.csv')
            except:
                print('Oh! We lost '+ str(i) + str(j))
                continue

# 上面有些沒有對「正確的股票簡稱」做正確的斷詞，在1_transformed_news作更正。

# 合併原始新聞＋斷詞後的新聞

In [5]:
for i in range(2013, 2022):
    for j in range(1,13):
        try:
            pd.read_csv('./202202_Intern/SegAndRawData/'+str(i)+str(j)+'_cnyesnews.csv')
        except:
            print(i, j)
            try:
                df_news_1 = pd.read_csv('./202202_Intern/RawData/'+str(i)+str(j)+'1_cnyesnews.csv')[['date', 'title', 'content']]
                df_news_2 = pd.read_csv('./202202_Intern/RawData/'+str(i)+str(j)+'2_cnyesnews.csv')[['date', 'title', 'content']]
                df_news = pd.concat([df_news_1, df_news_2]).reset_index(drop=True)
                df_news.date = pd.to_datetime(df_news.date).dt.date
                
                df_news['all'] = df_news['title']+df_news['content']
                df_news['all'] = df_news['all'].fillna('')
#                 df_news['segmented'] = df_news['all'].apply(lambda x : splitSentence(x))
#                 df_news = df_news[['date', 'segmented']]

                df_segnews = pd.read_csv('./202202_Intern/SegmentedData/'+str(i)+str(j)+'_cnyesnews.csv')
                df_news['segmented'] = df_segnews['segmented']
                df_news = df_news[['date', 'all', 'segmented']]

                df_news.to_csv('./202202_Intern/SegAndRawData/'+str(i)+str(j)+'_cnyesnews.csv', index=False)
            except:
                print('Oh! We lost '+ str(i) + str(j))
                continue